In [147]:
import os
import glob


def read_data(data_dir='data'):
    data = []
    
    path = os.path.join(data_dir, '*.txt')
    files = glob.glob(path)
    
    for f in files:
        with open(f,encoding="utf8") as document:
            data.append(document.read())
            
    return data

In [148]:
data = read_data()

In [149]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
import re

def document_to_words(document):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", document.lower())
    words = text.split()
    words = [w for w in words if w not in stopwords.words("english")]
    words = [PorterStemmer().stem(w) for w in words]
    
    return words

In [150]:
data_in_words = []
for document in data:
    data_in_words.append(document_to_words(document))

In [151]:
def term_frequency(data):
    word_count = {}
    for document in data:
        for word in document:
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1
    return word_count

In [152]:
freq = term_frequency(data_in_words)

In [161]:
def build_aux(data):
    word_document = {}
    for i in range (0,len(data)):
        for j in range (0,len(data[i])):
            if data[i][j] in word_document:
                word_document[data[i][j]][i].append(j)
            else:
                word_document[data[i][j]] = [[] for i in range(len(data))]
                word_document[data[i][j]][i].append(j)
    return word_document

In [166]:
aux = build_aux(data_in_words)

In [167]:
aux

{'object': [[0, 10, 90], [], [11], [], []],
 'orient': [[1], [], [], [], []],
 'program': [[2, 135], [], [], [], [4, 60, 67, 69, 73, 78, 82, 94, 95]],
 'inherit': [[3, 15, 26, 44, 81, 116, 125, 136, 151, 167, 168],
  [],
  [],
  [],
  []],
 'way': [[4], [270], [48], [64, 168], []],
 'form': [[5], [], [], [166], [65]],
 'new': [[6, 20, 162], [], [], [], []],
 'class': [[7, 12, 21, 24, 31, 34, 36, 89], [], [], [], []],
 'instanc': [[8, 91], [], [], [], [87]],
 'call': [[9, 84, 141], [40], [], [5, 139], [93, 265]],
 'use': [[11, 169],
  [4, 79, 95, 289],
  [18, 27],
  [19, 33],
  [24, 109, 129, 142, 155, 161, 197]],
 'alreadi': [[13], [], [], [], [245, 253, 261]],
 'defin': [[14], [155], [], [], []],
 'concept': [[16], [259], [], [], []],
 'invent': [[17], [286], [], [], []],
 '1967': [[18], [], [], [], []],
 'simula': [[19], [], [], [], []],
 'known': [[22, 62], [217], [52, 58], [], []],
 'deriv': [[23], [194], [], [142], []],
 'take': [[25], [], [], [128], [16]],
 'attribut': [[27], [],

In [172]:
sorted_aux = dict(sorted(aux.items()))

In [173]:
sorted_aux

{'0': [[], [183], [], [], []],
 '1': [[], [73], [94], [], [136, 146]],
 '10': [[], [184], [], [], []],
 '1940': [[], [], [], [], [25]],
 '1953': [[], [], [], [], [39]],
 '1967': [[18], [], [], [], []],
 '2': [[], [], [107], [43], [151, 181]],
 '2005': [[], [83], [], [], []],
 '285': [[], [56], [], [], []],
 '3': [[], [], [122], [], [160]],
 '336': [[], [84], [], [], []],
 '4': [[], [], [136], [], []],
 '6': [[], [55], [], [], []],
 '8': [[], [74], [], [], []],
 '999': [[], [57], [], [], []],
 'abstract': [[102], [], [], [], []],
 'academ': [[], [247], [], [], []],
 'accept': [[], [], [], [], [99]],
 'accomplish': [[153], [], [], [], []],
 'accord': [[], [], [], [73], []],
 'account': [[], [], [], [129], []],
 'act': [[], [], [], [160], []],
 'action': [[], [], [], [], [85, 101]],
 'actual': [[], [225], [], [], []],
 'ad': [[161], [], [], [], []],
 'adjac': [[], [], [], [], [127]],
 'advanc': [[], [], [], [], [288]],
 'advantag': [[124], [], [], [], []],
 'algebra': [[], [], [6], [], []